In [147]:
# !pip install pgeocode
# !pip install fastkml
# !pip install pykml

In [148]:
import pandas as pd
import missingno as msno
import numpy as np
import pgeocode
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import statistics
import geopy
from datetime import date, timedelta
import csv
from geopy.distance import geodesic
from geopy.distance import great_circle
dist = pgeocode.GeoDistance('US')
nomi = pgeocode.Nominatim('US')
from fastkml import kml
import os
from pykml import parser
from datetime import datetime
#from google.colab import drive

In [149]:
# drive.mount('/content/drive')

In [150]:
# Set the datapath

DataPath='C:/Users/ragarza2/OneDrive - Source Logistics/Documents/General/Project/Capstone/Predictive Part/'
DataPath2='C:/Users/ragarza2/OneDrive - Source Logistics/Documents/General/Project/Capstone/Daily Orders Part/Data/'
DataPath3='C:/Users/ragarza2/OneDrive - Source Logistics/Documents/General/Project/Capstone/Daily Orders Part/Data/'
DataPath4='C:/Users/ragarza2/OneDrive - Source Logistics/Documents/General/Project/Capstone/Clustering Part/Data/'

# DataPath = '/content/drive/My Drive/Colab Notebooks/Data/'
# DataPath2 = '/content/drive/MyDrive/Colab Notebooks/Data/Freight Consolidation Report/'
# DataPath3 = '/content/drive/My Drive/Colab Notebooks/Data/'

In [151]:
# Automate file naming with ending string representing code for todays date yymmdd

today = datetime.today().strftime('%y%m%d')

In [152]:
#import data

cluster=pd.read_csv(DataPath+'final.clust.res.csv')
#cons_rep=pd.read_csv(DataPath2+'Freigth_Consolidation_Report_' + today + '.csv', skiprows = 3)
cons_rep=pd.read_csv(DataPath2+'Freigth_Consolidation_Report_210616.csv', skiprows = 3)
#cons_ver=pd.read_csv(DataPath2+'Freight_Consolidation_Veracore_' + today + '.csv')
cons_ver=pd.read_csv(DataPath2+'Freight_Consolidation_Veracore_210616.csv')
new_code=pd.read_csv(DataPath4+'Phase 1 Company Codes_210811.csv')

In [153]:
# Set parameters

# Diameter for clusters between 50 to 100 miles from warehouse
first_diam = 10

# Diameter for clusters between 100 to 750 miles from warehouse
second_diam = 50

# Diameter for clusters beyond 750 miles from warehouse
third_diam = 100

# Margin of error for shipments belonging to a cluster
error = 0.20

# Number of past days to consider in analysis
days = 10

In [154]:
# Subset Velocity data to only select companies

searchfor = new_code['Older Code']
cons_rep = cons_rep[cons_rep['CUST_NBR_1'].str.contains('|'.join(searchfor), na=False, case=False)]

In [155]:
# Change old Velocity code to new code

df_dict = new_code.drop(['Company'], axis=1)
dict_code = df_dict.set_index('Older Code').T.to_dict('list')
cons_rep['New Code'] = cons_rep['CUST_NBR_1'].str.upper().map(dict_code)
cons_rep['New Code'] = cons_rep['New Code'].str[0].astype(np.int64)
cons_rep = cons_rep.drop(['CUST_NBR_1'], axis = 1)
cons_rep = cons_rep.rename({'New Code': 'CUST_NBR_1'}, axis=1) 

In [156]:
# Subset Veracore data to only select companies

cons_ver = cons_ver[cons_ver['OMS'].str.contains(r'[0-9]')]
cons_ver['OMS'] = cons_ver['OMS'].astype(str).astype(int)
searchfor = new_code['Master Code']
cons_ver = cons_ver[cons_ver['OMS'].isin(searchfor)]

In [157]:
# Visualize Veracore data to clean

#cons_ver.head()

In [158]:
# Rename zip code column to match name in Velocity data Set - used later on in the code
cons_ver.rename(columns={'Ship To-Zip/Postal Code':'SHIP_TO_POSTAL_CODE'}, inplace=True) 

# change the SHIP_TO_POSTAL_CODE type from object to string
cons_ver['SHIP_TO_POSTAL_CODE'] = cons_ver['SHIP_TO_POSTAL_CODE'].astype('str')

# Remove zip code that start with letters
cons_ver = cons_ver[~cons_ver['SHIP_TO_POSTAL_CODE'].str.contains("[a-zA-Z]").fillna(False)]

# Remove rows that is less than 4 digits
cons_ver['length'] = cons_ver['SHIP_TO_POSTAL_CODE'].apply(lambda x: len(x))
cond = cons_ver['length'] >3
cons_ver = cons_ver[cond]

#remove everything after'-' eg.(-3793)
cons_ver['SHIP_TO_POSTAL_CODE'] = cons_ver['SHIP_TO_POSTAL_CODE'].str.split('-').str[0]

# Extract first 5 digits from a Pandas column
cons_ver['SHIP_TO_POSTAL_CODE'] = cons_ver.SHIP_TO_POSTAL_CODE.str.extract(r'(\d{1,5})').squeeze().str.zfill(5)

# remove length column
cons_ver = cons_ver.drop(columns=['length'])

# remove Nan values in SHIP_TO_POSTAL_CODE
cons_ver = cons_ver[cons_ver['SHIP_TO_POSTAL_CODE'].notna()]

In [159]:
# Clean NeedByDate column, delete everything after letter "T"

cons_ver['NeedByDate'] = cons_ver['NeedByDate'].str.split('T').str[0]

In [160]:
# Filter out all with "S" (Shipped) in Order Status

cons_ver = cons_ver[~cons_ver['Order Status'].str.contains("S")]

In [161]:
# Filter the Velocity data set for only applicable orders for the consolidation program

cons_rep = cons_rep[(cons_rep['ORDER_STATUS_DESC_1']=='Submitted (Customer)')]

In [162]:
# Change column names for Velocity data (cons_rep)

cons_rep.rename(columns = {'CUST_NBR_1':'OMS', 'ORDER_NBR_1':'Order ID', 'CUSTOMER_PO_NBR':'Purchase Order', 
                           'ORDER_STATUS_DESC_1':'Order Status', 'ORDER_DATE':'Order Date', 'EXPIRATION_DATE':'NeedByDate',
                          'SHIP_METHOD_DESC':'Order Ship To Requested Freight Carrier', 'SHIP_TO_COMPANY':'Ship To-Company',
                          'SHIP_TO_ADDR_LINE_1':'Ship To-Address 1', 'SHIP_TO_CITY':'Ship To-City','SHIP_TO_STATE':'Ship To-State',
                          'QTY_ORDERED':'Qty Ordered', 'WEIGHT_PER_UNIT':'Total Weight','Pallets':'Total Pallets'}, inplace = True)

In [163]:
# Append Velocity data set to Veracore one

cons = cons_ver.append(cons_rep, ignore_index = True) 

C:\Users\ragarza2\AppData\Local\Temp/ipykernel_24492/300249494.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cons = cons_ver.append(cons_rep, ignore_index = True)


In [164]:
# Create unique identifier for orders by concatenating order id and date

cons['Unique ID'] = cons['Order Date'].astype('datetime64[ns]').apply(lambda x: datetime.strftime(x, '%y%m%d')) + cons['Order ID'].apply(str)

In [165]:
# Only consider orders placed a certain number of days ago to today

#time_lag = date.today() - timedelta(days)
time_lag = date(2021,6,16) - timedelta(days)
cons['Order Date']= pd.to_datetime(cons['Order Date'])
cons = cons[(cons['Order Date'] > pd.Timestamp(time_lag))]

In [166]:
# Drop A_Pro and PRINTED columns in Velocity data set as they are not critical

cons.drop(columns=['WMS', 'A_PRO', 'PRINTED'], inplace = True)

cons.drop(columns=['Product Owner Description', 'Ship To-Full Name', 'Ship To-Company'], inplace = True)

In [167]:
# Exclude certain orders based on Freight Carrier column

cons = cons[~cons['Order Ship To Requested Freight Carrier'].isin(['USPS', 'UPS', 'FEDEX', 'CONPU','Consignee Pick Up'])]

In [168]:
cons

,OMS,Order ID,Purchase Order,Reference Number,Order Status,Order Date,NeedByDate,Order Ship To Requested Freight Carrier,Ship To-Address 1,Ship To-City,Ship To-State,SHIP_TO_POSTAL_CODE,Qty Ordered,Total Weight,Total Pallets,Unnamed: 19,REQUEST_SHIP_DATE_1,Unique ID
20,101017,1693,30033,554193,U H,2021-06-07,1/1/2050,WILL CALL,1500 EASTRIDGE,RIVERSIDE,CA,92507,1311,21606.33,13.18,NaN,NaN,2106071693
21,101017,1713,136416,554705,P X,2021-06-10,6/14/2021,WILL CALL,9999 NIBLICK DRIVE,ROSEVILLE,CA,95678,512,5377.58,3.28,NaN,NaN,2106101713
22,101017,1714,76826,554734,U H,2021-06-10,1/1/2050,WILL CALL,1990 POMONA ROAD,CORONA,CA,92880,112,39200.00,28.00,NaN,NaN,2106101714
23,101017,1716,544980,554917,U H,2021-06-11,1/1/2050,WILL CALL,4199 GIBRALTAR COURT,STOCKTON,CA,95206,22,7700.00,5.50,NaN,NaN,2106111716
24,101017,1718,326049,554924,U H,2021-06-11,1/1/2050,WILL CALL,15510 CARMENITA ROAD,SANTA FE SPRINGS,CA,90670,52,35068.80,52.00,NaN,NaN,2106111718
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
296,101033,499011,MS10013416-02,NaN,Submitted (Customer),2021-06-10,NaN,Source Freight,14149 WEST MCDOWELL RD,GOODYEAR,AZ,85395,595,11118.00,7.00,NaN,6/23/2021,210610499011
297,101033,499122,QC0706ox,NaN,Submitted (Customer),2021-06-14,NaN,Source Freight,401 DEL NORTE BLVD,OXNARD,CA,93030,1700,32079.00,20.00,NaN,6/16/2021,210614499122
298,101033,499137,1HVVG2,NaN,Submitted (Customer),2021-06-14,NaN,Customer Pick Up,4193 INDUSTRIAL PARKWAY DRIVE,LEBEC,CA,93243,544,5037.44,3.20,NaN,6/18/2021,210614499137
299,101033,499163,BA1553-1,NaN,Submitted (Customer),2021-06-14,NaN,Source Freight,451 N. COTA STREET,CORONA,CA,92880,170,3145.00,2.00,NaN,6/16/2021,210614499163


In [169]:
# Read developed routes and save them in a dataframe for later use

dat_14=pd.read_csv(DataPath3+'Phoenix Route.csv')
dat_24=pd.read_csv(DataPath3+'Sacramento Route.csv')
dat_48=pd.read_csv(DataPath3+'Seattle Route.csv')
dat_38=pd.read_csv(DataPath3+'Denver Route.csv')
dat_12=pd.read_csv(DataPath3+'Salt Lake City Route.csv')

routes = [14,24,48,38,12]

dat_cons=pd.read_csv(DataPath3+'Locally Consolidated - LA.csv')

In [170]:
# Create coordinate dataframe from csv file

def coordtab(coord):
    coord.columns = ['Coordinates']
    coord[['Longitude','Latitude','Zero']] = coord['Coordinates'].str.split(',', expand=True)
    coord.drop(['Coordinates', 'Zero'], axis=1,inplace = True)
    coord['Longitude'] = coord.Longitude.astype(float)
    coord['Latitude'] = coord.Latitude.astype(float)
    coord = coord[['Latitude','Longitude']]
    coord['Coordinates'] = coord.values.tolist()
    return coord

In [171]:
# Create function to measure minimum distance between a point and a line (or lane)

def closest(points, new_point):
    closest_point = None
    closest_distance = None
    for point in points:
        distance = ((point[0] - new_point[0])**2 + (point[1] - new_point[1])**2)**0.5
        if closest_distance is None or distance < closest_distance:
            closest_point = point
            closest_distance = distance
    return closest_point

In [172]:
# Transform to usable format

#[14,24,48,38,12]

coord_12 = coordtab(dat_12)
coord_14 = coordtab(dat_14)
coord_24 = coordtab(dat_24)
coord_48 = coordtab(dat_48)
coord_38 = coordtab(dat_38)

coord_cons = coordtab(dat_cons)

C:\Users\ragarza2\AppData\Local\Temp/ipykernel_24492/2853645120.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  coord['Coordinates'] = coord.values.tolist()
C:\Users\ragarza2\AppData\Local\Temp/ipykernel_24492/2853645120.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  coord['Coordinates'] = coord.values.tolist()
C:\Users\ragarza2\AppData\Local\Temp/ipykernel_24492/2853645120.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexe

In [173]:
# Set facility zip code
origin_zip = 90640
latitude_origin_zip=nomi.query_postal_code(origin_zip).latitude
longitude_origin_zip=nomi.query_postal_code(origin_zip).longitude
location_origin_zip=(latitude_origin_zip,longitude_origin_zip)   

In [174]:
# Create unique zip code dataframe
zip_code = cons.SHIP_TO_POSTAL_CODE
zip_code.drop_duplicates(inplace = True)
zip_code = zip_code.reset_index(drop = True)
zip_code = pd.DataFrame(zip_code) 
zip_code.dropna(inplace = True)

In [175]:
# Create a function to identify whether zip codes lie inside the "locally consolidated" area and exclude them after.

# A Python3 program to check if a given point  
# lies inside a given polygon 
# Refer https://www.geeksforgeeks.org/check-if-two-given-line-segments-intersect/ 
# for explanation of functions onSegment(), 
# orientation() and doIntersect()  
 
# Define Infinite (Using INT_MAX  
# caused overflow problems)
INT_MAX = 10000
 
# Given three colinear points p, q, r,  
# the function checks if point q lies 
# on line segment 'pr' 
def onSegment(p:tuple, q:tuple, r:tuple) -> bool:
     
    if ((q[0] <= max(p[0], r[0])) &
        (q[0] >= min(p[0], r[0])) &
        (q[1] <= max(p[1], r[1])) &
        (q[1] >= min(p[1], r[1]))):
        return True
         
    return False
 
# To find orientation of ordered triplet (p, q, r). 
# The function returns following values 
# 0 --> p, q and r are colinear 
# 1 --> Clockwise 
# 2 --> Counterclockwise 
def orientation(p:tuple, q:tuple, r:tuple) -> int:
     
    val = (((q[1] - p[1]) *
            (r[0] - q[0])) -
           ((q[0] - p[0]) *
            (r[1] - q[1])))
            
    if val == 0:
        return 0
    if val > 0:
        return 1 # Collinear
    else:
        return 2 # Clock or counterclock
 
def doIntersect(p1, q1, p2, q2):
     
    # Find the four orientations needed for  
    # general and special cases 
    o1 = orientation(p1, q1, p2)
    o2 = orientation(p1, q1, q2)
    o3 = orientation(p2, q2, p1)
    o4 = orientation(p2, q2, q1)

    # General case
    if (o1 != o2) and (o3 != o4):
        return True

    # Special Cases 
    # p1, q1 and p2 are colinear and 
    # p2 lies on segment p1q1 
    if (o1 == 0) and (onSegment(p1, p2, q1)):
        return True

    # p1, q1 and p2 are colinear and 
    # q2 lies on segment p1q1 
    if (o2 == 0) and (onSegment(p1, q2, q1)):
        return True

    # p2, q2 and p1 are colinear and 
    # p1 lies on segment p2q2 
    if (o3 == 0) and (onSegment(p2, p1, q2)):
        return True

    # p2, q2 and q1 are colinear and 
    # q1 lies on segment p2q2 
    if (o4 == 0) and (onSegment(p2, q1, q2)):
        return True

    return False
 
# Returns true if the point p lies  
# inside the polygon[] with n vertices 
def is_inside_polygon(points:list, p:tuple) -> bool:
     
    n = len(points)
     
    # There must be at least 3 vertices
    # in polygon
    if n < 3:
        return False
         
    # Create a point for line segment
    # from p to infinite
    extreme = (INT_MAX, p[1])
    count = i = 0
     
    while True:
        next = (i + 1) % n
         
        # Check if the line segment from 'p' to  
        # 'extreme' intersects with the line  
        # segment from 'polygon[i]' to 'polygon[next]' 
        if (doIntersect(points[i],
                        points[next], 
                        p, extreme)):
                             
            # If the point 'p' is colinear with line  
            # segment 'i-next', then check if it lies  
            # on segment. If it lies, return true, otherwise false 
            if orientation(points[i], p, 
                           points[next]) == 0:
                return onSegment(points[i], p, 
                                 points[next])
                                  
            count += 1
             
        i = next
         
        if (i == 0):
            break
         
    # Return true if count is odd, false otherwise 
    return (count % 2 == 1)

In [176]:
zip_code['clust_range'] = ""
zip_code['close_clust'] = ""
zip_code['dist_to_clust'] = ""
zip_code['belongs_clust?'] = ""
#zip_code['dist_from_origin'] = ""
zip_code['inside_polygon?'] = ""

polygon = coord_cons['Coordinates']

for j in range(zip_code.shape[0]):
    try:
        # Step 1. transfer zip_code into geo data(Get the latitude and longitude based on the zip code
        latitude_new_zip=nomi.query_postal_code(zip_code.iloc[j][0]).latitude
        longitude_new_zip=nomi.query_postal_code(zip_code.iloc[j][0]).longitude
        location_new_zip=(latitude_new_zip,longitude_new_zip)

        # Step 1.1 Get distance from origin 
        #dist_orig=great_circle(location_origin_zip, location_new_zip).miles
        #zip_code.loc[j,'dist_from_origin'] = dist_orig

        # Step 2: Compare distance between orders in the system and centroids
        list=[]
        for i in range(cluster.shape[0]):
            a=great_circle(location_new_zip, (cluster.Centroid_lat[i],cluster.Centroid_long[i])).miles
            list.append(a)

        # Get the closest cluster number
        min_row = np.argmin(list)
        closest_cluster=cluster.iloc[min_row][0]

        # Get the cluster information from the dictionary
        #clust_range = clust_dict[clust_dict['cluster_label']==closest_cluster].iloc[0]['distance range']
        clust_range = cluster.iloc[min_row]['distance.range']

        # Determine if they are close enough?
        # Now check the distance between the closest cluster and the new zip_code
        dist = list[min_row]

        # Prepare zip code lat, long point to check if zip code is inside locally consolidated polygon
        p = [latitude_new_zip, longitude_new_zip]

        #If not
        # Add NA to the column
        zip_code.loc[j,'clust_range'] = clust_range
        zip_code.loc[j,'close_clust'] = closest_cluster
        zip_code.loc[j,'dist_to_clust'] = dist
        zip_code.loc[j,'inside_polygon?'] = 'Yes' if is_inside_polygon(points = polygon, p = p) else 'No'

        if clust_range == 'less than 50 miles from warehouse':
            zip_code.loc[j,'belongs_clust?'] = 'NA'
            
        elif clust_range == '50 to 100 miles from warehouse':
            if dist <= first_diam/2:
                zip_code.loc[j,'belongs_clust?'] = 'Yes'
            elif dist <= (1+error)*first_diam/2:
                zip_code.loc[j,'belongs_clust?'] = 'Maybe'
            else: 
                zip_code.loc[j,'belongs_clust?'] = 'No'
                
        elif clust_range == '100 to 750 miles from warehouse':
            if dist <= second_diam/2:
                zip_code.loc[j,'belongs_clust?'] = 'Yes'
            elif dist <= (1+error)*second_diam/2:
                zip_code.loc[j,'belongs_clust?'] = 'Maybe'
            else:
                zip_code.loc[j,'belongs_clust?'] = 'No'
                
        else:
            if dist <= third_diam/2:
                zip_code.loc[j,'belongs_clust?'] = 'Yes'
            elif dist <= (1+error)*third_diam/2:
                zip_code.loc[j,'belongs_clust?'] = 'Maybe'
            else:
                zip_code.loc[j,'belongs_clust?'] = 'No'
                
    except:
        pass

In [177]:
zip_code

,SHIP_TO_POSTAL_CODE,clust_range,close_clust,dist_to_clust,belongs_clust?,inside_polygon?
0,92507,100 to 750 miles from warehouse,14,297.925361,No,Yes
1,95678,100 to 750 miles from warehouse,24,11.919962,Yes,No
2,92880,100 to 750 miles from warehouse,14,313.147234,No,Yes
3,95206,100 to 750 miles from warehouse,13,19.438406,Yes,No
4,90670,100 to 750 miles from warehouse,13,309.120135,No,Yes
...,...,...,...,...,...,...
72,95340,100 to 750 miles from warehouse,13,45.20622,No,No
73,85395,100 to 750 miles from warehouse,14,13.05021,Yes,No
74,93030,100 to 750 miles from warehouse,20,262.582154,No,No
75,93243,100 to 750 miles from warehouse,13,230.932449,No,No


In [178]:
# Merge the cons and zip_code dataframes

cons['close_clust'] = 0
cons['dist_to_clust'] = 0
cons['belongs_clust?'] = 0
cons['inside_polygon?'] = 0

for i in range(cons.shape[0]):
    index = np.where(zip_code['SHIP_TO_POSTAL_CODE'] == cons.iloc[i]['SHIP_TO_POSTAL_CODE'])[0][0]
    cons.iloc[i, cons.columns.get_loc('close_clust')] = zip_code.iloc[index]['close_clust']
    cons.iloc[i, cons.columns.get_loc('dist_to_clust')] = zip_code.iloc[index]['dist_to_clust']
    cons.iloc[i, cons.columns.get_loc('belongs_clust?')] = zip_code.iloc[index]['belongs_clust?']
    cons.iloc[i, cons.columns.get_loc('inside_polygon?')] = zip_code.iloc[index]['inside_polygon?']

In [179]:
# Create dataframe of all those zip codes that didn't belong to a cluster to check if they're on the way instead.

on_way = zip_code.loc[(zip_code['belongs_clust?']!='Yes') & (zip_code['inside_polygon?']=='No')]

In [180]:
# Get the closest lanes by shipment and distance from such lane

index = range(len(on_way['clust_range']))
columns = ['SHIP_TO_POSTAL_CODE','closest_lane','closest_lat','closest_long','closest_distance']
on_way_df = pd.DataFrame(index=index, columns=columns)
on_way_df = on_way_df.fillna(0)

for i in range(len(on_way['clust_range'])):
     
    # Step 1. Transfer zip_code into geo data(Get the latitude and longitude based on the zip code
    latitude_on_way=nomi.query_postal_code(on_way.iloc[i][0]).latitude
    longitude_on_way=nomi.query_postal_code(on_way.iloc[i][0]).longitude
    location_on_way=(latitude_on_way,longitude_on_way)

    index = range(len(routes))
    columns = ['lat','long','distance']
    points = pd.DataFrame(index=index, columns=columns)
    points = points.fillna(0)

    for j in range(len(routes)):
        a = globals()["coord_"+str(routes[j])]
        b = closest(a['Coordinates'],location_on_way)
        d = great_circle(location_on_way, b).miles
        points.loc[j,'lat'] = b[0]
        points.loc[j,'long'] = b[1]
        points.loc[j,'distance'] = d
        
    # Get the closest cluster number
    min_row = np.argmin(points['distance'])
    closest_cluster=routes[min_row]

    # Store values in df
    on_way_df.loc[i,'SHIP_TO_POSTAL_CODE'] = on_way.iloc[i][0]
    on_way_df.loc[i,'closest_lane'] = closest_cluster
    on_way_df.loc[i,'closest_lat'] = points.iloc[min_row]['lat']
    on_way_df.loc[i,'closest_long'] = points.iloc[min_row]['long']
    on_way_df.loc[i,'closest_distance'] = points.iloc[min_row]['distance']

In [219]:
# Merge dataframes cons and on_way_df

final_df = pd.merge(left=cons, right=on_way_df, how='left', on='SHIP_TO_POSTAL_CODE')

In [220]:
# Round pallets to the next integer up if decimal is .1 or larger, otherwise round down
    # Except when less than 1, in which case always round up

def roundly(x):
    if (x >= 1):
        if ((x % 1) >= 0.1):
            return np.ceil(x)
        else:
            return np.floor(x)
    else:
        return np.ceil(x)

In [221]:
# Filter for orders outside of local consolidation area, belonging to a cluster and larger than 1,000 lbs

final_df = final_df[final_df['inside_polygon?']!='Yes']
final_df.drop(columns = ['inside_polygon?'],inplace = True)
#final_df = final_df[final_df['belongs_clust?']!='No']
#final_df.drop(columns = ['belongs_clust?'],inplace = True)
final_df = final_df[final_df['Total Weight']>250]
final_df = final_df[~((final_df['closest_distance']>30) & (final_df['belongs_clust?']=='No'))]
final_df['Total Pallets'] = final_df['Total Pallets'].apply(roundly)
final_df['Actual Weight'] = final_df['Total Pallets']*40 + final_df['Total Weight']
#final_df = final_df[(final_df['closest_distance']<30)|(final_df['closest_distance'].isnull())]

In [222]:
# Filter for data from today and last work day

dates = pd.to_datetime(final_df['Order Date'].dt.strftime('%Y-%m-%d'))
last_day = dates.drop_duplicates().nlargest(10).iloc[-1]
final_df = final_df[final_df['Order Date'] >= last_day]

In [223]:
# Visualize Results

final_df.drop(columns = ['closest_lat','closest_long','Unnamed: 19'],inplace=True)
final_df = final_df.round({"dist_to_clust":1, "Actual Weight":1, "closest_distance":1})
final_df.head(10)

,OMS,Order ID,Purchase Order,Reference Number,Order Status,Order Date,NeedByDate,Order Ship To Requested Freight Carrier,Ship To-Address 1,Ship To-City,...,Total Weight,Total Pallets,REQUEST_SHIP_DATE_1,Unique ID,close_clust,dist_to_clust,belongs_clust?,closest_lane,closest_distance,Actual Weight
1,101017,1713,136416,554705,P X,2021-06-10,6/14/2021,WILL CALL,9999 NIBLICK DRIVE,ROSEVILLE,...,5377.58,4.0,NaN,2106101713,24,11.9,Yes,NaN,NaN,5537.6
3,101017,1716,544980,554917,U H,2021-06-11,1/1/2050,WILL CALL,4199 GIBRALTAR COURT,STOCKTON,...,7700.00,6.0,NaN,2106111716,13,19.4,Yes,NaN,NaN,7940.0
10,101017,1731,153615600,555417,U H,2021-06-15,1/1/2050,WILL CALL,400 S. WOODLAND AVE,WOODBURN,...,2293.24,2.0,NaN,2106151731,43,11.8,Yes,NaN,NaN,2373.2
11,101017,1734,61395,554962,P,2021-06-15,6/17/2021,WILL CALL,697 LENFEST ROAD,SAN JOSE,...,989.31,1.0,NaN,2106151734,20,5.1,Yes,NaN,NaN,1029.3
12,101081,373,1.04152E+12,NaN,U H,2021-06-10,1/1/2050,CUSPU,"1761 W. INTERCHANGE DR, BLDG C",SAN BERNANDINO,...,14073.60,12.0,NaN,210610373,14,302.6,No,38.0,1.1,14553.6
13,101081,375,1.04204E+12,NaN,P,2021-06-10,6/16/2021,CUSPU,"1761 W. INTERCHANGE DR, BLDG C",SAN BERNANDINO,...,9382.40,8.0,NaN,210610375,14,302.6,No,38.0,1.1,9702.4
14,101081,376,1.04204E+12,NaN,U H,2021-06-10,1/1/2050,CUSPU,"1761 W. INTERCHANGE DR, BLDG C",SAN BERNANDINO,...,28147.20,24.0,NaN,210610376,14,302.6,No,38.0,1.1,29107.2
15,101081,377,1.04204E+12,NaN,U H,2021-06-10,1/1/2050,CUSPU,"1761 W. INTERCHANGE DR, BLDG C",SAN BERNANDINO,...,5864.00,5.0,NaN,210610377,14,302.6,No,38.0,1.1,6064.0
17,101017,1736,402218700,555428,U H,2021-06-15,1/1/2050,WILL CALL,4400 CROWS LANDING RD,MODESTO,...,2066.30,2.0,NaN,2106151736,13,6.3,Yes,NaN,NaN,2146.3
19,101017,1739,137074,555423,U H,2021-06-15,1/1/2050,WILL CALL,9999 NIBLICK DRIVE,ROSEVILLE,...,5377.58,4.0,NaN,2106151739,24,11.9,Yes,NaN,NaN,5537.6


In [186]:
final_df.to_csv('final_df_' + today + '.csv')
#final_df.to_csv('final_df_210325.csv')